In [1]:
import pickle

In [2]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_ML100k'
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small'

In [3]:
MYMEDIALITEPATH='/home/kirana/MyMediaLite/bin'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import sklearn as sklearn
import datetime as dt 
import seaborn as sns
import torch as torch

In [5]:
from sklearn import metrics

In [6]:
[dftrain,dfvalid]=pickle.load(open(f'{DATAPATH}/df_side_cold.pkl','rb'))

In [7]:
dftrain['rating'].describe()

count    90543.000000
mean         3.503269
std          1.040251
min          0.500000
25%          3.000000
50%          3.500000
75%          4.000000
max          5.000000
Name: rating, dtype: float64

In [8]:
dfinp_train=dftrain[['userId','itemId','rating']]
dfinp_test=dfvalid[['userId','itemId','rating']]
dfinp_train.to_csv(f'{PATH}/inp_train_ml100k.csv',index=False,header=False,sep=",")
dfinp_test.to_csv(f'{PATH}/inp_test_ml100k.csv',index=False,header=False,sep=",")

In [9]:
TRAINFILEPATH=f'{PATH}/inp_train_ml100k.csv'
TESTFILEPATH=f'{PATH}/inp_test_ml100k.csv'

In [10]:
dfinp_train.head()

,userId,itemId,rating
0,1,804,4.0
1,1,1210,5.0
2,1,2018,5.0
3,1,2628,4.0
4,1,2826,4.0


### Global Average

In [58]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=GlobalAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_globalavg.txt'

In [59]:
!command {cmd}

=loading_time 0.13
memory 6
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
GlobalAverage  training_time 00:00:00.0007530 RMSE 1.062371 MAE 0.8417616 CBD 0.2773427 new users: RMSE 1.29238 MAE 1.21391 CBD 0.21794 new items: RMSE 1.17591 MAE 0.93992 CBD 0.29655 testing_time 00:00:00.0123730
prediction_time 00:00:00.0114900

memory 6
>

In [60]:
predictions_globalavg=pd.read_csv (f'{DATAPATH}/prediction_globalavg.txt',sep='\t',header=None).iloc[:,2]

In [61]:
dfvalid['pred_globalavg']=predictions_globalavg

In [64]:
def score_by_group (dfvalid,predVar,groupVar='cold_start_group',method='global_avg'):
    scores=[]
    mae_scores=[]
    r2_scores=[]
    groups=dfvalid[groupVar].value_counts().index.tolist()
    for group in groups:
        df=dfvalid.loc[dfvalid[groupVar]==group]
        actual=df['rating']
        score=metrics.mean_squared_error(df['rating'],df[predVar])
        mae_score=metrics.mean_absolute_error(df['rating'],df[predVar])
        r2_score=metrics.r2_score(df['rating'],df[predVar])
        scores.append(score)
        mae_scores.append(mae_score)
        r2_scores.append(r2_score)
    return pd.DataFrame({'method':method, 'mse':scores, 'rmse':np.sqrt(scores), 'mae':mae_scores, 'r2_score':r2_scores, 'Cold Start Group':groups})

In [66]:
result_globalavg=pd.concat([score_by_group(dfvalid,'pred_globalavg'),score_by_group(dfvalid,'pred_globalavg','cold_start_bucket'),\
score_by_group(dfvalid,'pred_globalavg','flag_train')],axis=0)

In [67]:
result_globalavg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,global_avg,1.103706,1.050574,0.830738,-0.000002,No-cold-start
1,global_avg,1.382762,1.175909,0.939917,-0.070920,Item-Cold_start
2,global_avg,1.670246,1.292380,1.213910,-0.818685,User-Cold-Start
0,global_avg,1.103706,1.050574,0.830738,-0.000002,0
1,global_avg,1.410617,1.187694,0.966464,-0.026058,1
0,global_avg,1.128633,1.062371,0.841762,-0.000249,0


In [68]:
dfvalid['flag_train'].value_counts()

0    10293
Name: flag_train, dtype: int64

### User Average

In [69]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_useravg.txt'

In [73]:
!command {cmd}

=loading_time 0.13
memory 6
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
UserAverage  training_time 00:00:00.0038480 RMSE 0.9638703 MAE 0.7517028 CBD 0.2677646 new users: RMSE 1.29238 MAE 1.21391 CBD 0.21794 new items: RMSE 0.92753 MAE 0.72625 CBD 0.27224 testing_time 00:00:00.0123610
prediction_time 00:00:00.0113570

memory 6
>

In [74]:
predictions_useravg=pd.read_csv (f'{DATAPATH}/prediction_useravg.txt',sep='\t',header=None).iloc[:,2]

In [75]:
dfvalid['pred_useravg']=predictions_useravg

In [78]:
result_useravg=pd.concat([score_by_group(dfvalid,'pred_useravg','cold_start_group','useravg'),score_by_group(dfvalid,'pred_useravg','cold_start_bucket','useravg'),\
score_by_group(dfvalid,'pred_useravg','flag_train','useravg')],axis=0)
result_useravg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,useravg,0.928184,0.963423,0.749776,0.159027,No-cold-start
1,useravg,0.860319,0.927534,0.726245,0.333701,Item-Cold_start
2,useravg,1.670246,1.292380,1.213910,-0.818685,User-Cold-Start
0,useravg,0.928184,0.963423,0.749776,0.159027,0
1,useravg,0.938793,0.968913,0.773495,0.317138,1
0,useravg,0.929046,0.963870,0.751703,0.176634,0


### Item Average

In [79]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemavg.txt'

In [83]:
!command {cmd}

=loading_time 0.13
memory 6
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
ItemAverage  training_time 00:00:00.0038060 RMSE 1.005098 MAE 0.7822351 CBD 0.2729497 new users: RMSE 1.07006 MAE 0.94056 CBD 0.18865 new items: RMSE 1.17591 MAE 0.93992 CBD 0.29655 testing_time 00:00:00.0124650
prediction_time 00:00:00.0112360

memory 6
>

In [84]:
predictions_itemavg=pd.read_csv (f'{DATAPATH}/prediction_itemavg.txt',sep='\t',header=None).iloc[:,2]

In [85]:
dfvalid['pred_itemavg']=predictions_itemavg

In [86]:
result_itemavg=pd.concat([score_by_group(dfvalid,'pred_itemavg','cold_start_group','itemavg'),score_by_group(dfvalid,'pred_itemavg','cold_start_bucket','itemavg'),\
score_by_group(dfvalid,'pred_itemavg','flag_train','itemavg')],axis=0)
result_itemavg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemavg,0.979327,0.989609,0.768291,0.112690,No-cold-start
1,itemavg,1.382762,1.175909,0.939917,-0.070920,Item-Cold_start
2,itemavg,1.145031,1.070061,0.940559,-0.246793,User-Cold-Start
0,itemavg,0.979327,0.989609,0.768291,0.112690,0
1,itemavg,1.359729,1.166074,0.939979,0.010957,1
0,itemavg,1.010223,1.005098,0.782235,0.104691,0


## Neighborhood Methods

### Item KNN Cosine

In [87]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemKNN \
--test-file={TESTFILEPATH} --prediction-file {PATH}/prediction_itemknn_cosine.txt \
--recommender-options k=40  --recommender-options reg_u=12 \
--recommender-options reg_i=1'

In [88]:
!command {cmd}

=loading_time 0.14
memory 10
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
ItemKNN k=40 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=12 reg_i=1 num_iter=10  training_time 00:01:35.9596140 RMSE 0.8622037 MAE 0.6608837 CBD 0.2589309 new users: RMSE 1.04586 MAE 0.90926 CBD 0.18598 new items: RMSE 0.98521 MAE 0.78885 CBD 0.27767 testing_time 00:01:53.3945720
prediction_time 00:01:53.6390810

memory 457
>

In [90]:
predictions_itemknn_cosine=pd.read_csv (f'{PATH}/prediction_itemknn_cosine.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_itemknn_cosine']=predictions_itemknn_cosine

In [92]:
result_itemknncosine=pd.concat([score_by_group(dfvalid,'pred_itemknn_cosine','cold_start_group','itemknncosine'),score_by_group(dfvalid,'pred_itemknn_cosine','cold_start_bucket','itemknncosine'),\
score_by_group(dfvalid,'pred_itemknn_cosine','flag_train','itemknncosine')],axis=0)
result_itemknncosine

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemknncosine,0.722252,0.849854,0.648540,0.345610,No-cold-start
1,itemknncosine,0.970639,0.985210,0.788848,0.248261,Item-Cold_start
2,itemknncosine,1.093818,1.045858,0.909257,-0.191028,User-Cold-Start
0,itemknncosine,0.722252,0.849854,0.648540,0.345610,0
1,itemknncosine,0.982574,0.991249,0.800515,0.285293,1
0,itemknncosine,0.743395,0.862204,0.660884,0.341167,0


### Item KNN Pearson

In [98]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemKNN \
--test-file={TESTFILEPATH} --prediction-file {PATH}/prediction_itemknn_pearson.txt \
--recommender-options k=40  --recommender-options reg_u=12 \
--recommender-options reg_i=1 --recommender-options correlation=Pearson'

In [99]:
!command {cmd}

=loading_time 0.15
memory 10
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
ItemKNN k=40 correlation=Pearson weighted_binary=False alpha=0; baseline predictor: reg_u=12 reg_i=1 num_iter=10  training_time 00:00:10.0009720 RMSE 0.8921673 MAE 0.6859663 CBD 0.2616013 new users: RMSE 1.04586 MAE 0.90926 CBD 0.18598 new items: RMSE 0.98521 MAE 0.78885 CBD 0.27767 testing_time 00:00:36.7668910
prediction_time 00:00:35.5415710

memory 457
>

In [100]:
predictions_itemknn_pearson=pd.read_csv (f'{PATH}/prediction_itemknn_pearson.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_itemknn_pearson']=predictions_itemknn_pearson

In [101]:
result_itemknnpearson=pd.concat([score_by_group(dfvalid,'pred_itemknn_pearson','cold_start_group','itemknnpearson'),score_by_group(dfvalid,'pred_itemknn_pearson','cold_start_bucket','itemknnpearson'),\
score_by_group(dfvalid,'pred_itemknn_pearson','flag_train','itemknnpearson')],axis=0)
result_itemknnpearson

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemknnpearson,0.779466,0.882874,0.675840,0.293772,No-cold-start
1,itemknnpearson,0.970639,0.985210,0.788848,0.248261,Item-Cold_start
2,itemknnpearson,1.093818,1.045858,0.909257,-0.191028,User-Cold-Start
0,itemknnpearson,0.779466,0.882874,0.675840,0.293772,0
1,itemknnpearson,0.982574,0.991249,0.800515,0.285293,1
0,itemknnpearson,0.795963,0.892167,0.685966,0.294579,0


### User KNN Pearson

In [110]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserKNN \
--test-file={TESTFILEPATH} --prediction-file {PATH}/prediction_userknn_pearson.txt \
--recommender-options k=60  --recommender-options reg_u=12 \
--recommender-options reg_i=1 --recommender-options correlation=Pearson'

In [111]:
!command {cmd}

=loading_time 0.13
memory 8
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
UserKNN k=60 correlation=Pearson weighted_binary=False alpha=0; baseline predictor: reg_u=12 reg_i=1 num_iter=10  training_time 00:00:00.2327230 RMSE 0.9021646 MAE 0.6933887 CBD 0.263685 new users: RMSE 1.04586 MAE 0.90926 CBD 0.18598 new items: RMSE 0.98521 MAE 0.78885 CBD 0.27767 testing_time 00:00:06.0826140
prediction_time 00:00:06.0391380

memory 12
>

In [112]:
predictions_userknn_pearson=pd.read_csv (f'{PATH}/prediction_userknn_pearson.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_userknn_pearson']=predictions_userknn_pearson

In [113]:
result_userknnpearson=pd.concat([score_by_group(dfvalid,'pred_userknn_pearson','cold_start_group','userknnpearson'),score_by_group(dfvalid,'pred_userknn_pearson','cold_start_bucket','userknnpearson'),\
score_by_group(dfvalid,'pred_userknn_pearson','flag_train','userknnpearson')],axis=0)
result_userknnpearson

,method,mse,rmse,mae,r2_score,Cold Start Group
0,userknnpearson,0.798990,0.893863,0.683919,0.276082,No-cold-start
1,userknnpearson,0.970639,0.985210,0.788848,0.248261,Item-Cold_start
2,userknnpearson,1.093818,1.045858,0.909257,-0.191028,User-Cold-Start
0,userknnpearson,0.798990,0.893863,0.683919,0.276082,0
1,userknnpearson,0.982574,0.991249,0.800515,0.285293,1
0,userknnpearson,0.813901,0.902165,0.693389,0.278681,0


### User KNN Cosine

In [106]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserKNN \
--test-file={TESTFILEPATH} --prediction-file {PATH}/prediction_userknn_cosine.txt \
--recommender-options k=40  --recommender-options reg_u=12 \
--recommender-options reg_i=1'

In [107]:
!command {cmd}

=loading_time 0.14
memory 8
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
UserKNN k=40 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=12 reg_i=1 num_iter=10  training_time 00:00:00.8910720 RMSE 0.9017445 MAE 0.6925191 CBD 0.2634306 new users: RMSE 1.04586 MAE 0.90926 CBD 0.18598 new items: RMSE 0.98521 MAE 0.78885 CBD 0.27767 testing_time 00:00:09.3640610
prediction_time 00:00:09.2501880

memory 12
>

In [117]:
predictions_userknn_cosine=pd.read_csv (f'{PATH}/prediction_userknn_cosine.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_userknn_cosine']=predictions_userknn_cosine

In [109]:
result_userknncosine=pd.concat([score_by_group(dfvalid,'pred_userknn_cosine','cold_start_group','userknncosine'),score_by_group(dfvalid,'pred_userknn_cosine','cold_start_bucket','userknncosine'),\
score_by_group(dfvalid,'pred_userknn_cosine','flag_train','userknncosine')],axis=0)
result_userknncosine

,method,mse,rmse,mae,r2_score,Cold Start Group
0,userknncosine,0.798165,0.893401,0.682972,0.276830,No-cold-start
1,userknncosine,0.970639,0.985210,0.788848,0.248261,Item-Cold_start
2,userknncosine,1.093818,1.045858,0.909257,-0.191028,User-Cold-Start
0,userknncosine,0.798165,0.893401,0.682972,0.276830,0
1,userknncosine,0.982574,0.991249,0.800515,0.285293,1
0,userknncosine,0.813143,0.901744,0.692519,0.279353,0


## Matrix Factorization

### SVD

In [124]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=MatrixFactorization \
--test-file={TESTFILEPATH} --prediction-file {PATH}/prediction_svd.txt \
--recommender-options num_factors=10  --recommender-options regularization=0.1 \
--recommender-options learn_rate=0.07 --recommender-options num_iter=100'

In [125]:
!command {cmd}

=loading_time 0.12
memory 6
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
MatrixFactorization num_factors=10 regularization=0.1 learn_rate=0.07 learn_rate_decay=1 num_iter=100  training_time 00:00:01.9950760 RMSE 0.9359761 MAE 0.7165015 CBD 0.2654603 new users: RMSE 1.29238 MAE 1.21391 CBD 0.21794 new items: RMSE 1.17591 MAE 0.93992 CBD 0.29655 testing_time 00:00:00.0095820
prediction_time 00:00:00.0112550

memory 7
>

In [118]:
predictions_svd=pd.read_csv (f'{PATH}/prediction_svd.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_svd']=predictions_svd

In [119]:
result_svd=pd.concat([score_by_group(dfvalid,'pred_svd','cold_start_group','svd'),score_by_group(dfvalid,'pred_svd','cold_start_bucket','svd'),\
score_by_group(dfvalid,'pred_svd','flag_train','svd')],axis=0)
result_svd

,method,mse,rmse,mae,r2_score,Cold Start Group
0,svd,0.831569,0.911904,0.696508,0.246565,No-cold-start
1,svd,1.382762,1.175909,0.939917,-0.070920,Item-Cold_start
2,svd,1.670246,1.292380,1.213910,-0.818685,User-Cold-Start
0,svd,0.831569,0.911904,0.696508,0.246565,0
1,svd,1.410617,1.187694,0.966464,-0.026058,1
0,svd,0.878599,0.937336,0.718434,0.221342,0


### SVD Plus Plus

In [129]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=SVDPlusPlus \
--test-file={TESTFILEPATH} --prediction-file {PATH}/prediction_svdplusplus.txt \
--recommender-options num_factors=50  --recommender-options regularization=1 \
--recommender-options learn_rate=0.01 --recommender-options num_iter=50 \
--recommender-options bias_reg=0.005 --recommender-options bias_learn_rate=0.07 \
--recommender-options frequency_regularization=True'



In [130]:
cmd

'/home/kirana/MyMediaLite/bin/rating_prediction --training-file=/home/kirana/Documents/phd/experiment1_new/DNNREC_ML100k/inp_train_ml100k.csv --recommender=SVDPlusPlus --test-file=/home/kirana/Documents/phd/experiment1_new/DNNREC_ML100k/inp_test_ml100k.csv --prediction-file /home/kirana/Documents/phd/experiment1_new/DNNREC_ML100k/prediction_svdplusplus.txt --recommender-options num_factors=50  --recommender-options regularization=1 --recommender-options learn_rate=0.01 --recommender-options num_iter=50 --recommender-options bias_reg=0.005 --recommender-options bias_learn_rate=0.07 --recommender-options frequency_regularization=True'

In [131]:
!command {cmd}

=loading_time 0.13
memory 6
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
SVDPlusPlus num_factors=50 regularization=1 bias_reg=0.005 frequency_regularization=True learn_rate=0.01 bias_learn_rate=0.07 learn_rate_decay=1 num_iter=50  training_time 00:26:05.1397950 RMSE 0.8694718 MAE 0.6714504 CBD 0.2591881 new users: RMSE 1.13105 MAE 1.03757 CBD 0.19751 new items: RMSE 0.9452 MAE 0.74998 CBD 0.27379 testing_time 00:00:00.0177370
prediction_time 00:00:00.0124020

memory 11
>

In [132]:
predictions_svdplusplus=pd.read_csv (f'{PATH}/prediction_svdplusplus.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_svdplusplus']=predictions_svdplusplus

result_svdplusplus=pd.concat([score_by_group(dfvalid,'pred_svdplusplus','cold_start_group','svdplusplus'),score_by_group(dfvalid,'pred_svdplusplus','cold_start_bucket','svdplusplus'),\
score_by_group(dfvalid,'pred_svdplusplus','flag_train','svdplusplus')],axis=0)
result_svdplusplus

,method,mse,rmse,mae,r2_score,Cold Start Group
0,svdplusplus,0.740528,0.860539,0.662045,0.329052,No-cold-start
1,svdplusplus,0.893407,0.945202,0.749977,0.308076,Item-Cold_start
2,svdplusplus,1.279266,1.131047,1.037567,-0.392958,User-Cold-Start
0,svdplusplus,0.740528,0.860539,0.662045,0.329052,0
1,svdplusplus,0.930793,0.964776,0.777842,0.322958,1
0,svdplusplus,0.755981,0.869472,0.671450,0.330013,0


### BiasedMatrixFactorization

In [136]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=BiasedMatrixFactorization \
--test-file={TESTFILEPATH} --prediction-file {PATH}/prediction_biasedmatrixfactorization.txt \
--recommender-options num_factors=40   --recommender-options bias_reg=0.1 \
--recommender-options reg_u=1   --recommender-options reg_i=1.2 \
--recommender-options learn_rate=0.07 --recommender-options num_iter=100 \
--recommender-options bold_driver=True \
--recommender-options frequency_regularization=True'



In [137]:
!command {cmd}

=loading_time 0.12
memory 6
ratings range: [0.5, 5]
training data: 607 users, 9012 items, 90543 ratings, sparsity 98.34482
test data:     584 users, 4479 items, 10293 ratings, sparsity 99.6065
BiasedMatrixFactorization num_factors=40 bias_reg=0.1 reg_u=1 reg_i=1.2 frequency_regularization=True learn_rate=0.07 bias_learn_rate=1 learn_rate_decay=1 num_iter=100 bold_driver=True loss=RMSE max_threads=1 naive_parallelization=False objective 66344.3359375 learn_rate 0.0735 
objective 60975.890625 learn_rate 0.077175 
objective 54387.453125 learn_rate 0.08103374 
objective 48152.8359375 learn_rate 0.08508543 
objective 43884.3984375 learn_rate 0.0893397 
objective 41055.125 learn_rate 0.09380668 
objective 39156.32421875 learn_rate 0.098497 
objective 37896.60546875 learn_rate 0.1034219 
objective 37101.3203125 learn_rate 0.1085929 
objective 36656.70703125 learn_rate 0.1140226 
objective 36486.87890625 learn_rate 0.1197237 
objective 36542.15625 learn_rate 0.05986185 
objective 32488.947265

In [138]:
predictions_biasedmatrixfactorization=pd.read_csv (f'{PATH}/prediction_biasedmatrixfactorization.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_biasedmatrixfactorization']=predictions_biasedmatrixfactorization

result_biasedmatrixfactorization=pd.concat([score_by_group(dfvalid,'pred_biasedmatrixfactorization','cold_start_group','biasedmatrixfactorization'),score_by_group(dfvalid,'pred_biasedmatrixfactorization','cold_start_bucket','biasedmatrixfactorization'),\
score_by_group(dfvalid,'pred_biasedmatrixfactorization','flag_train','biasedmatrixfactorization')],axis=0)
result_biasedmatrixfactorization

,method,mse,rmse,mae,r2_score,Cold Start Group
0,biasedmatrixfactorization,0.747125,0.864364,0.662981,0.323074,No-cold-start
1,biasedmatrixfactorization,0.997892,0.998946,0.796329,0.227154,Item-Cold_start
2,biasedmatrixfactorization,1.099959,1.048789,0.920787,-0.197715,User-Cold-Start
0,biasedmatrixfactorization,0.747125,0.864364,0.662981,0.323074,0
1,biasedmatrixfactorization,1.007782,1.003883,0.808388,0.266957,1
0,biasedmatrixfactorization,0.768296,0.876525,0.674791,0.319099,0


In [139]:
results_concat=pd.concat ([result_biasedmatrixfactorization,
result_svdplusplus,
result_svd,
result_userknncosine,
result_userknnpearson,
result_itemknnpearson,
result_itemknncosine,
result_itemavg,
result_useravg,
result_globalavg],axis=0)

In [141]:
pickle.dump(results_concat,open(f'{DATAPATH}/diff_results.pkl','wb'))

In [142]:
results_concat

,method,mse,rmse,mae,r2_score,Cold Start Group
0,biasedmatrixfactorization,0.747125,0.864364,0.662981,0.323074,No-cold-start
1,biasedmatrixfactorization,0.997892,0.998946,0.796329,0.227154,Item-Cold_start
2,biasedmatrixfactorization,1.099959,1.048789,0.920787,-0.197715,User-Cold-Start
0,biasedmatrixfactorization,0.747125,0.864364,0.662981,0.323074,0
1,biasedmatrixfactorization,1.007782,1.003883,0.808388,0.266957,1
0,biasedmatrixfactorization,0.768296,0.876525,0.674791,0.319099,0
0,svdplusplus,0.740528,0.860539,0.662045,0.329052,No-cold-start
1,svdplusplus,0.893407,0.945202,0.749977,0.308076,Item-Cold_start
2,svdplusplus,1.279266,1.131047,1.037567,-0.392958,User-Cold-Start
0,svdplusplus,0.740528,0.860539,0.662045,0.329052,0


In [144]:
pickle.dump([dftrain,dfvalid,results_concat],open(f'{DATAPATH}/df_side_cold_runother.pkl','wb'))